In [ ]:
## to run in colab

# !git clone https://github.com/sergeychuvakin/advanced_nlp_course.git
# !mv advanced_nlp_course/LM/*.py ./
# !pip install loguru pydantic tokenizers

In [16]:
%reload_ext autoreload
%autoreload 2

from torch.utils.data import DataLoader
from loguru import logger
import sys


from dependencies import corpus, tokenizer
from config import Config, LanguageModelConfig
from processing_utils import (
    clean_text, 
    split_on_sequences, 
    create_ngrams, 
    create_to_x_and_y, 
    word2int,
    create_vocab
)
from model import LM_LSTM
from datahandler import LMDataset
from train_utils import train_model

config = Config()

logger.remove()
logger.add(sys.stderr, level="WARNING")

7

In [ ]:
corpus = clean_text(corpus)
corpus = split_on_sequences(corpus)

tcorpus = tuple(map(lambda sentence: tokenizer.encode(sentence), corpus))


## create n-grams for each doc
sq = create_ngrams(tcorpus, config.N_GRAM)

## shift corpus to create x and y 
x, y =  create_to_x_and_y(sq)

token_id, id_token = create_vocab(tcorpus)
vocab_size = len(token_id)

## for passing to dataloader
x_int = [word2int(i, token_id) for i in x]
y_int = [word2int(i, token_id) for i in y]

## split data
tradeoff_index = int(len(x_int) * config.TRAIN_PROPORTION)

x_train = x_int[:tradeoff_index]
x_test = x_int[tradeoff_index:]

y_train = y_int[:tradeoff_index]
y_test = y_int[tradeoff_index:]

logger.warning(f"Outpur shapes: x_train: {len(x_train)}, x_test: {len(x_test)}, y_train: {len(y_train)}, y_test: {len(y_test)}")

## load to dataset and dataloader
train_ds = LMDataset(x_train, y_train)
test_ds = LMDataset(x_test, y_test)

train_dl = DataLoader(train_ds, batch_size=config.BATCH_SIZE, shuffle=True)
test_dl = DataLoader(test_ds, batch_size=config.BATCH_SIZE, shuffle=False)

# model and model config
model_config = LanguageModelConfig(vocab_size=vocab_size, emb_size=300)
model = LM_LSTM(**model_config.dict(), logger=logger)


optimizer = torch.optim.Adam(model.parameters(), lr=model_config.lr)
loss_func = torch.nn.CrossEntropyLoss()

# train model 
tmodel = train_model(model,
                     train_dl,
                     optimizer=optimizer,
                     loss_func=loss_func,
                     batch_size=config.BATCH_SIZE,
                     epochs=30, 
                     clip=1)

# torch.save(model.state_dict(), config.SAVE_MODEL_FNAME)

100%|██████████| 52810/52810 [00:00<00:00, 145790.56it/s]
2021-11-10 21:15:24.926 | WARNING  | __main__:<module>:29 - Outpur shapes: x_train: 969530, x_test: 157831, y_train: 969530, y_test: 157831
  0%|          | 2/970 [00:02<21:18,  1.32s/it]

Cross-entropy loss: 9.452272415161133



  0%|          | 3/970 [00:03<20:57,  1.30s/it]

Cross-entropy loss: 9.452272415161133



  0%|          | 4/970 [00:05<20:50,  1.29s/it]

...

Cross-entropy loss: 9.386908531188965



 11%|█         | 106/970 [02:25<18:09,  1.26s/it]

Cross-entropy loss: 9.382908821105957



 11%|█         | 107/970 [02:27<18:01,  1.25s/it]

Cross-entropy loss: 9.377412796020508


In [ ]:
model_config.dict()